### FILE MANAGER
Esta ferramenta gerencia arquivos nas pastas usadas no CPDOC, separando os 20 arquivos de documentos mais correlacionados com cada tópico,  juntando-os em pastas específicas. Isso permite que se avalie a qualidade do processo de topic modelling.

In [1]:
import os
import sys
import re
from IPython.display import clear_output
import shutil
import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import csv
import xlrd
import xlwt

In [2]:
path_input = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/'
path_output = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/topic modelling/analise_de_documentos/'

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Verifica qual sistema operacional está sendo usado. Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'

In [ ]:
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = ssh_priv_key,
                        ssh_private_key_password = pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura dados na base mysql. Itera ao longo de tópicos.
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    for topic in range(0,100):
        cur.execute('''SELECT * FROM (SELECT doc_id, topic_id, topic_score, docs.readability 
        FROM topic_doc
        JOIN docs
        ON docs.id = doc_id WHERE topic_id = "%s") AS doc_id_leg''' % topic) 
        data = cur.fetchall()
        subfolder = 'topic_'+str(topic)+'/'
        
        ### Checa se subpasta já existe ###
        if os.path.exists(path_output+subfolder):
            print('ERROR DETECTED\r\nSubfolder already exists. To run this code, please delete manually all subfolders at: \r\n'+path_output)
            break
        else: 
            os.makedirs(path_output+subfolder)
        
        ### Copia arquivos para análise e salva info sobre cada documento em arquivo do Excel ###
        workbook = xlwt.Workbook()
        info = 'info_topic'+str(topic)
        worksheet = workbook.add_sheet('info')
        worksheet.write(0, 0, "doc_id")
        worksheet.write(0, 1, "topic_id")
        worksheet.write(0, 2, "topic_score")
        worksheet.write(0, 3, "readability")
        worksheet.write(0, 4, "cohesion")
        
        for row in data[0:20]:
            doc = row[0] 
            count = -1
            for cell in row:
                count += 1
                worksheet.write(data.index(row)+1, count, cell)
            
            copy_file = path_input+'AAS_mre_'+doc+'.txt'
            paste_file = path_output+subfolder+'AAS_mre_'+doc+'.txt'
            shutil.copy2(copy_file, paste_file)
        workbook.save(path_output+subfolder+info+'.xls')
    

········
········
info_topic0
(('bp_1974.06.04_doc_I-8', 0, 0.802068, Decimal('1.00')), ('ag_1974.01.22_doc_III-55', 0, 0.710061, Decimal('0.85')), ('rb_1974.04.17_doc_I-25', 0, 0.695807, Decimal('0.88')), ('rb_1974.05.23_doc_III-11', 0, 0.689216, Decimal('0.85')), ('rb_1974.05.23_doc_III-22', 0, 0.688374, Decimal('1.00')), ('ag_1974.01.22_doc_II-28', 0, 0.671601, Decimal('0.96')), ('ag_1974.01.22_doc_III-29', 0, 0.648689, Decimal('1.00')), ('ag_1974.01.22_doc_III-44', 0, 0.642961, Decimal('0.93')), ('pn_1974.09.11_doc_3', 0, 0.632175, Decimal('0.89')), ('rb_1974.12.13_doc_4', 0, 0.628623, Decimal('0.89')), ('rb_1974.03.19_doc_I-23', 0, 0.628007, Decimal('0.80')), ('rb_1974.03.19_doc_I-33', 0, 0.627308, Decimal('0.91')), ('rb_1974.08.19_doc_I-54', 0, 0.623483, Decimal('0.97')), ('rb_1974.10.24_doc_13', 0, 0.619064, Decimal('1.00')), ('ag_1974.01.16_doc_I-10', 0, 0.61771, Decimal('0.95')), ('rb_1974.02.06_doc_I-34', 0, 0.610483, Decimal('0.85')), ('ag_1974.01.16_doc_I-8', 0, 0.591277, D